In [ ]:
import itertools
import pickle
from pathlib import Path

import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from gammapy.estimators import FluxPoints
from gammapy.modeling.models import SkyModel, create_crab_spectral_model

In [ ]:
# Plot style and axis format
style.use("seaborn-colorblind")

sns.set_color_codes("colorblind")
sns.set_palette("colorblind")

MPL_LINEWIDTH = 1.6

mpl_rc = {
    "figure.autolayout": False,
    "font.size": 24,
    "figure.dpi": 300,
    "lines.linewidth": 2.7,
    "lines.markersize": 7,
    "axes.grid": False,
    "axes.linewidth": MPL_LINEWIDTH,
    "xtick.major.size": 8,
    "xtick.major.width": MPL_LINEWIDTH,
    "xtick.minor.size": 5,
    "xtick.minor.width": MPL_LINEWIDTH,
    "xtick.minor.visible": False,
    "ytick.major.size": 8,
    "ytick.major.width": MPL_LINEWIDTH,
    "ytick.minor.size": 5,
    "ytick.minor.width": MPL_LINEWIDTH,
    "ytick.minor.visible": True,
}
plt.style.use(mpl_rc)


crab_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [0.05, 30] * u.TeV,
    "yunits": u.Unit("erg cm-2 s-1"),
    "linestyle": "--",
}

sed_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [0.05, 30] * u.TeV,
}
sed_kwargs_joint_fit = {
    "sed_type": "e2dnde",
    "energy_bounds": [0.02, 2] * u.TeV,
}


def set_axes(axis):
    """Set axes limits and labels."""
    axis.set_xlim(1e-3, 4e1)
    axis.set_ylim(1e-12, 3e-10)

    axis.set_xlabel(r"$E\,\,[{\rm TeV}]$")
    axis.set_ylabel(
        r"$E^2 \frac{{\rm d}\phi}{{\rm d} E}\, [{\rm erg}\,{\rm cm}^{-2}\,{\rm s}^{-1}]$"
    )

In [ ]:
# Files
BASE_PATH = Path("data")
src_indep = BASE_PATH / "src_indep"
fermi_lat_sed_file = BASE_PATH / "SED_Crab_FermiLAT_Arakawa2020.fits"
lst1_only_sed_model_file = src_indep / "SED_model_CrabNebula_only_LST1.dat"
lst1_fermi_joint_model_file = src_indep / "SED_model_CrabNebula_joint_LST1_FermiLAT.dat"
lst1_flux_points_file = src_indep / "SED_CrabNebula_LST1_flux_points.fits"

In [ ]:
def plot_fermi_ref(axis):
    """Plot spectrum from Fermi-LAT (Arakawa et al. 2020)."""
    fermi_flux_points = FluxPoints.read(fermi_lat_sed_file)

    fermi_flux_points.plot(
        label="$Fermi$-LAT (Arakawa et al. 2020)",
        color="m",
        marker="s",
        zorder=30,
        sed_type="e2dnde",
        ax=axis,
        markersize=6,
    )


def plot_magic_ref(axis):
    """Plot reference spectrum from MAGIC (Aleksić et al. 2015)."""
    create_crab_spectral_model("magic_lp").plot(
        **crab_kwargs,
        label="MAGIC (Aleksić et al. 2015)",
        color="r",
        ax=axis,
        zorder=20,
    )

In [ ]:
with open(lst1_only_sed_model_file, "rb") as flux_model:
    lst1_only_model_dict = pickle.load(flux_model)
    lst1_only_model = SkyModel.from_dict(lst1_only_model_dict)


with open(lst1_fermi_joint_model_file, "rb") as flux_model:
    join_model_dict = pickle.load(flux_model)
    joint_model = SkyModel.from_dict(join_model_dict)


lst1_flux_points = FluxPoints.read(lst1_flux_points_file, hdu="SED", format="gadf-sed")


def plot_band_several_efficiencies(axis):
    """Plot SED band from several efficiencies in the gammaness and theta selection cuts."""
    gamma_efficiencies = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    theta_efficiencies = [0.7, 0.8, 0.9]

    sed_kwargs_hatch = {
        "sed_type": "e2dnde",
        "energy_bounds": [50 * u.GeV, 30 * u.TeV],
        "facecolor": "w",
        "edgecolor": "b",
        "hatch": "//",
        "alpha": 1,
    }

    for count, (gh_eff, th_eff) in enumerate(
        itertools.product(gamma_efficiencies, theta_efficiencies)
    ):
        file_model = (
            src_indep
            / f"effi_band/model_stacked_8bin_r2914_to_r7277_gh_eff_{gh_eff}_th_cont_{th_eff}.dat"
        )
        flux_points = (
            src_indep
            / f"effi_band/sed_flux_points_8bin_r2914_to_r7277_gh_eff_{gh_eff}_th_cont_{th_eff}.fits"
        )

        # Open files
        with open(file_model, "rb") as flux_model:
            model_dict = pickle.load(flux_model)

        flux_points = FluxPoints.read(
            flux_points,
            hdu="SED",
            format="gadf-sed",
            reference_model=SkyModel.from_dict(model_dict),
        )

        if count == 0:
            flux_points.reference_model.spectral_model.plot_error(
                **sed_kwargs_hatch,
                label="Uncertainty from several efficiency cuts (LST-1)",
                ax=axis,
            )
        else:
            flux_points.reference_model.spectral_model.plot_error(
                **sed_kwargs_hatch, ax=axis
            )


def plot_test_background_normalization(axis):
    """Plot SED band"""
    flux_points_table_sys = FluxPoints.read(
        "data/src_indep/test_background_sys_LST1.fits"
    )

    flux_points_table_sys.plot(
        ax=axis,
        sed_type="e2dnde",
        markerfacecolor="white",
        color="lightgray",
        marker="o",
        xerr=None,
        label=r"$+1\%$ background systematics test (LST-1)",
        zorder=99,
        markeredgewidth=2.4,
    )

In [ ]:
# Make the plot
fig, ax = plt.subplots(1, 1, figsize=(18.4, 10))  # 12, 10 for the others

lst1_flux_points.plot(
    sed_type="e2dnde",
    ax=ax,
    label="LST-1 (this work)",
    xerr=None,
    color="black",
    zorder=100,
)

lst1_only_model.spectral_model.plot(
    **sed_kwargs, ax=ax, label="Log-parabola fit (LST-1)"
)
lst1_only_model.spectral_model.plot_error(
    **sed_kwargs,
    ax=ax,
    facecolor="b",
    alpha=0.3,
    label="Stat. uncertainty (LST-1)",
    zorder=8,
)

joint_model.spectral_model.plot(
    **sed_kwargs_joint_fit,
    ax=ax,
    label="Log-parabola joint fit ($Fermi$-LAT and LST-1)",
    linestyle=":",
    color="g",
)
joint_model.spectral_model.plot_error(**sed_kwargs_joint_fit, ax=ax)

# Plot MAGIC and Fermi-LAT SEDs
plot_fermi_ref(ax)
plot_magic_ref(ax)

# Test several efficiencies
plot_band_several_efficiencies(ax)
# Test a change of +1% in background normalization
plot_test_background_normalization(ax)

# Format axes
set_axes(ax)

# Reorder the labels in the legend
handles, labels = plt.gca().get_legend_handles_labels()
order = [5, 0, 1, 7, 4, 2, 3, 6]
plt.legend(
    [handles[idx] for idx in order],
    [labels[idx] for idx in order],
    loc="lower left",
    fontsize=23,
)

plt.savefig("lst1_crab_2023_performance_repo.png", bbox_inches="tight")
plt.savefig("lst1_crab_2023_performance_repo.pdf", bbox_inches="tight")